<a href="https://colab.research.google.com/github/saqib-sarwar/mt/blob/main/Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving eng_Hindi_data_dev_X.csv to eng_Hindi_data_dev_X.csv
Saving eng_Hindi_data_train.csv to eng_Hindi_data_train.csv


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
import pandas as pd

column_names = ["", "."]
df = pd.read_csv("/content/eng_Hindi_data_train.csv", names=column_names)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140000 entries, 0 to 139999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0           140000 non-null  object
 1   .       140000 non-null  object
dtypes: object(2)
memory usage: 2.1+ MB


In [ ]:
df.iloc[0]

     and deliver us by Thy mercy from the people of...
.    और अपनी रहमत से हमें इन काफ़िर लोगों (के नीचे)...
Name: 0, dtype: object

In [ ]:
df_test = pd.read_csv("/content/eng_Hindi_data_dev_X.csv", names=[""])
df_test.head()

,
0,और अनुसर्ण करो उस सर्वोत्तम चीज़ का जो तुम्हार...
1,एक क़ाफ़िला आया। फिर उसने पनिहारा को भेजा। उसन...
2,जो कोई सुचरित लेकर आया उसको उससे भी अच्छा प्रा...
3,raviratlami @aol. inEMAIL OF TRANSLATORS
4,घटना/कार्य/बैठक संपादक में RSVP क्षेत्र दिखायें


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0           40000 non-null  object
dtypes: object(1)
memory usage: 312.6+ KB


In [ ]:
# Sort the training data based on string  length

sorted_index = df[""].str.len().sort_values().index
df_sorted = df.reindex(sorted_index)
df_sorted = df_sorted.reset_index(drop=True)

In [ ]:
df_sorted.head()

,,.
0,% 1 / s,"% 1/s% 1 is the label, we add a ':' to it"
1,0 N 0 W,0 N 0 W
2,% 1 +% 2,% 1 +% 2
3,% 1 #% 2,% 1% 2An unknown transport type
4,% s h% s,% s h% s


In [ ]:
df_sorted.head(10)

,,.
0,% 1 / s,"% 1/s% 1 is the label, we add a ':' to it"
1,0 N 0 W,0 N 0 W
2,% 1 +% 2,% 1 +% 2
3,% 1 #% 2,% 1% 2An unknown transport type
4,% s h% s,% s h% s
5,% 1 -% 2,% 1 से% 2
6,% s =% s,% s =% s
7,of M ā g,मोर काIndian National month 12-ShortNamePosses...
8,% 1 -% 2,"% 1-% 2% 1: Start Date,% 2: Start Time"
9,% 1 @% 2,% 1 @% 2


In [ ]:
df_sorted.sample()

,,.
7779,Close _ without Saving,बिना सहेजे बंद करें (_ w)


In [ ]:
# Store the training data as a txt file with tab separated sentence pairs
df_sorted.to_csv('eng_hin_train_sorted.txt', sep='\t', index=False)

In [ ]:
# Sort test data

sorted_index_test = df_test[""].str.len().sort_values().index
df_test_sorted = df_test.reindex(sorted_index_test)
df_test_sorted = df_test_sorted.reset_index(drop=True)

In [ ]:
df_test_sorted.head(10)

,
0,% s h% s
1,% s h% s
2,% s =% s
3,% 1 x% 2
4,% s h% s
5,% s =% s
6,% s =% s
7,% 1 (% 2)
8,% A% d% B
9,% s (% s)


In [ ]:
df_test_sorted.to_csv('eng_hin_dev_sorted.txt', sep='\t', index=False)

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def normalizeHindi(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[a-zA-Z0-9%=()+-/*@]+", r" ", s)
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('%s_%s_train_sorted.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
#    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    pairs = []
    for l in lines:
      spl = l.split("\t")
      e,h = "",""
      for s in spl:
        if (spl.index(s) == 0):
          e = normalizeString(s)
        else:
          h = normalizeHindi(s)
      pairs.append([e,h])

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 50

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'hin', True)
print(random.choice(pairs))

Reading lines...
Read 140001 sentence pairs
Trimmed to 122260 sentence pairs
Counting words...
Counted words:
hin 18138
eng 19507
['यह वह दिन ह कि व कछ बोल नही रह ह ', 'this is a day in which they will not be able to speak .']


**VOCABULARY**

**MODEL**

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

**Hyperparameters**

**TRAINING**

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

**EVALUATION**

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

2m 57s (- 41m 27s) (5000 6%) 5.1781
5m 54s (- 38m 23s) (10000 13%) 4.8915
9m 3s (- 36m 15s) (15000 20%) 4.7165
12m 13s (- 33m 36s) (20000 26%) 4.5343
15m 19s (- 30m 38s) (25000 33%) 4.4389
18m 22s (- 27m 34s) (30000 40%) 4.2764
21m 26s (- 24m 30s) (35000 46%) 4.1897
24m 30s (- 21m 27s) (40000 53%) 4.1165
27m 33s (- 18m 22s) (45000 60%) 4.0778
30m 40s (- 15m 20s) (50000 66%) 4.0220
33m 45s (- 12m 16s) (55000 73%) 3.9631
36m 49s (- 9m 12s) (60000 80%) 3.8921
39m 51s (- 6m 7s) (65000 86%) 3.8287
42m 56s (- 3m 4s) (70000 93%) 3.7720
45m 58s (- 0m 0s) (75000 100%) 3.7667


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

> छवियो को बरन करन क लिए तलाश करन क दौरान बराउज किया गया पिछला फोलडर
= the last browsed folder while looking for images to burn
< the for to use for the used to use the <EOS>

> """   "" म डिसक को फिर लोड किए जान की जररत ह  "
=  the disc in s needs to be reloaded . 
<  the disc is in the disc .  <EOS>

> जो अपराधी ह व ईमान लानवालो पर हसत थ 
= indeed the guilty used to laugh at the believers .
< those who believe and will be in a . the <EOS>

> जिस दिन लोग भयकर चीख को सतयतः सन रह होग। वही दिन होगा निकलन का। 
= on the day they hear the cry in truth that is the day of coming forth .
< the day when the will be the the the the the the the the the . <EOS>

>  इसी तरह कौम  समद न पगमबरो को झठलाया
= the thamud accused the messengers of lies .
< the the denied the the denied the the <EOS>

> गतवय फोलडर की पषटि हत पछ बगर बनाए
= create destination folder without asking confirmation
< create a search for folder s <EOS>

> फाइल सची परापत कर रह
= getting the file list
< get list of <EOS>

> असथायी फा

In [ ]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "अपराधी ह व ईमान लानवालो पर हसत ")
plt.matshow(attentions.numpy())

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("समृद्ध पाठ के भाग के रूप में प्रदर्शित करें")

evaluateAndShowAttention("संदेश फलक के नीचे")

evaluateAndShowAttention("पूर्ण स्क्रीन मोड में एवींस चलायें")

evaluateAndShowAttention("में जुड़ नहीं सकाः")

KeyError: ignored

In [ ]:
from torchtext.data.metrics import bleu_score

#bleu_score(candidate_corpus, references_corpus)

**TESTING**